# Spark assignment 2: Collocations

As for the second part of the assignment, your task is to extract collocations: that is word combinations that occur together. For example, “high school” or “roman empire”.

To find collocations, you will use NPMI (normalized pointwise mutual information) metric.

PMI of two words, a & b, is defined as “PMI(a, b) = ln (P(ab) / (P(a) * P(b))”, where P(ab) is the probability of two words coming one after the other, and P(a) and P(b) are probabilities of words a & b respectively.

You will estimate probabilities with occurrence counts, that is “P(a) = # of occurrences of word a / total number of words”, and “P(ab) = # of occurrences of words ‘a b’ / total number of word pairs”.

To build an intuition behind the definition, see Reading material.

Therefore, rare combinations of coupled words have large PMI.

NPMI is computed as “NPMI(a, b) = PMI(a, b) / -ln P(ab)”. This normalizes the quantity to be within the range [-1; 1].

You task is a bit more complicated now:

- Extract all the words, as in the previous task.
- Filter out stopwords using the dictionary (/datasets/stop_words_en.txt ) (do not forget to convert words to the lowercase!)
- Compute all bigrams (that is, pairs of consequent words)
- Leave only bigrams with at least 500 occurrences
- Compute NPMI for every bigram (note: when computing probabilities, you need unpruned counts!)
- Sort word pairs by NPMI in the descending order
- Print top 39 word pairs, with words delimited by the underscore “_”

For example,

    roman_empire
    south_africa

The part of the result on the sample dataset:

    ...
    references_reading
    notes_references
    award_best
    north_america
    new_zealand
    ...
 
Hint: if you did everything right, “roman_empire” and “south_africa” are going to be in the result.

If you want to deploy the environment on your own machine, please use bigdatateam/spark-course1 Docker container.

In [1]:
#! /usr/bin/env python

from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local[2]"))

import re
import math

stop_file = "/datasets/stop_words_en.txt"
wiki_file = "/data/wiki/en_articles_part/articles-part"
pair_thresh = 500

In [2]:
#a=sc.parallelize([1, 2, 3, 4, 5])
#a.collect()

In [3]:
with open(stop_file, "r") as f:
    stop_words = f.read().splitlines()
    
stop_words_bcast = sc.broadcast(stop_words)

In [4]:
def parse_article(line):
    try:
        article_id, text = line.rstrip().split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        return words
    except ValueError as e:
        return []
    
def lower(words):
    return [word.lower() for word in words]

def filter_stop(words):
    return [word for word in words if word not in stop_words_bcast.value]

def pairs(words):
    out = []
    for w1, w2 in zip(words[:-1], words[1:]):
        out.append((w1.lower() + "_" + w2.lower(), 1))
    return out

In [5]:
wiki = (sc.textFile(wiki_file, 16)
         .map(parse_article) 
         .map(lower)
         .map(filter_stop)
        ).cache()

In [6]:
print wiki.count()

4100


In [7]:
words = (wiki.flatMap(lambda wds : [(word, 1) for word in wds])
         .reduceByKey(lambda x,y : x+y)
        ).cache()

pairs = (wiki.flatMap(pairs)
         .reduceByKey(lambda x,y : x+y)
        ).cache()

In [8]:
for key, count in words.take(10):
    print("%s\t%d" % (key, count))

biennials	10
underlyingly	1
ancyra	43
tripolitan	2
tilton	4
nordland	1
squealer	8
regularize	2
skylights.passive	1
thesis"(kleene	1


In [9]:
pairs.take(10)

[(u'2,000_1.5', 1),
 (u'fastest_mode', 1),
 (u'cases_federal', 4),
 (u'creem_particular', 1),
 (u'subgroups_lie', 2),
 (u'defendiendo_chile', 1),
 (u'vol_62', 4),
 (u'initial_production', 7),
 (u'buffalo_niagaras', 1),
 (u'flames_bas-reliefs', 1)]

In [22]:
words_count = words.map(lambda value: value[1]).sum()
words_countb = sc.broadcast(words_count)
words_count_map = words.collectAsMap()
words_count_mapb = sc.broadcast(words_count_map)

pairs_count = pairs.map(lambda value: value[1]).sum()
pairs_countb = sc.broadcast(pairs_count)
pairs_count_map = pairs.collectAsMap()

In [26]:
print words_count, pairs_count, words_countb.value, pairs_countb.value
print words_count_map.get("ancyra")
print pairs_count_map.get("cases_federal")

6971026 6966926 6971026 6966926
43
4


## Building an intuition behind PMI

PMI of two words, a & b, is defined as “PMI(a, b) = ln (P(ab) / (P(a) * P(b))”, where P(ab) is the probability of two words coming one after the other, and P(a) and P(b) are probabilities of words a & b respectively.

You will estimate probabilities with occurrence counts, that is “P(a) = # of occurrences of word a / total number of words”, and “P(ab) = # of occurrences of words ‘a b’ / total number of word pairs”.

To build an intuition behind the definition, consider the following cases:

__“roman empire”__; assume that this is a unique combination, and every occurrence of “roman” is followed by “empire”, and, vice versa, every occurrence of “empire” is preceded by “roman”. In this case, “P(ab) = P(a) = P(b)”, so “PMI(a, b) = -ln P(a) = -ln P(b)”. This quantity increases when the probability of the collocation is low.

__“the doors”__; let’s assume that “the” may occur with every word, independently. Thus, “P(ab) = P(a)*P(b)”, and “PMI(a, b) = ln 1 = 0”.

__“green idea / sleeps furiously”__; when two words never occur together, “P(ab) = 0”, and “PMI(a, b) = -inf”. Therefore, rare combinations of coupled words have large PMI.

If you want to deploy the environment on your own machine, please use [bigdatateam/spark-course1](https://hub.docker.com/r/bigdatateam/spark-course1/) Docker container.

In [35]:
def npmi(value):
    pair, count = value
    w1, w2 = pair.split("_")
    w1_count = words_count_mapb.value[w1]
    w2_count = words_count_mapb.value[w2]
    
    prob_pair = float(count) / pairs_countb.value
    prob_w1 = float(w1_count) / words_countb.value
    prob_w2 = float(w2_count) / words_countb.value
    pmi = math.log(prob_pair / (prob_w1 * prob_w2))
    npmi = pmi / (-1 * math.log(prob_pair))
    return (pair, npmi)

#PMI(a, b) = ln (P(ab) / (P(a) * P(b))
#P(a) = # of occurrences of word a / total number of words
#P(ab) = # of occurrences of words ‘a b’ / total number of word pairs
#NPMI = NPMI(a, b) = PMI(a, b) / -ln P(ab)

In [36]:
result = (pairs.filter(lambda value : value[1] > 500)
            .map(lambda value: npmi(value))
            .sortBy(lambda value: value[1], ascending=False)
            ).cache()

In [45]:
for key, value in result.take(39):
    print ("%s\t%.2f" % (key, value))

los_angeles	0.97
external_links	0.95
united_states	0.88
prime_minister	0.88
san_francisco	0.85
et_al	0.80
new_york	0.79
supreme_court	0.78
19th_century	0.76
20th_century	0.75
references_external	0.73
soviet_union	0.73
air_force	0.71
baseball_player	0.69
university_press	0.69
roman_catholic	0.68
united_kingdom	0.68
references_reading	0.67
notes_references	0.66
award_best	0.66
north_america	0.65
new_zealand	0.65
civil_war	0.64
catholic_church	0.63
world_war	0.62
war_ii	0.62
south_africa	0.62
took_place	0.61
roman_empire	0.61
united_nations	0.61
american_singer-songwriter	0.57
high_school	0.56
american_actor	0.56
american_actress	0.54
american_baseball	0.51
york_city	0.49
american_football	0.48
years_later	0.42
north_american	0.38


In [39]:
print result.take(10)

[(u'los_angeles', 0.9728997994856813), (u'external_links', 0.9496902298660744), (u'united_states', 0.8833319251733903), (u'prime_minister', 0.8827431167372655), (u'san_francisco', 0.8522919430661895), (u'et_al', 0.8025243317394334), (u'new_york', 0.7870688931579025), (u'supreme_court', 0.7781367974409652), (u'19th_century', 0.7574641661772452), (u'20th_century', 0.7514604533485757)]
